In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
import time
import numpy as np
import seaborn as sns

In [515]:
class board:
    
    files = ['a','b','c','d','e','f','g','h']
    ranks = [1,2,3,4,5,6,7,8]
    moveDict = {'upx':('up','x'),
               'downx':('down','x'),
               'leftx':('left','x'),
               'rightx':('right','x'),
               'upxleftx':(('up','x'),('left','x'),2),
               'upxrightx':(('up','x'),('right','x'),2),
               'downxleftx':(('down','x'),('left','x'),2),
               'downxrightx':(('down','x'),('right','x'),2),
               'up1':('up',1),
               'down1':('down',1),
               'left1':('left',1),
               'right1':('right',1),
               'up1left1':(('up',1),('left',1),2),
               'up1right1':(('up',1),('right',1),2),
               'down1left1':(('down',1),('left',1),2),
               'down1right1':(('down',1),('right',1),2),
               'up2left':(('up',2),('left',1),2),
               'up2right':(('up',2),('right',1),2),
               'left2up':(('left',2),('up',1),2),
               'left2down':(('left',2),('down',1),2),
               'right2up':(('right',2),('up',1),2),
               'right2down':(('right',2),('down',1),2),
               'down2left':(('down',2),('left',1),2),
               'down2right':(('down',2),('right',1),2),
               'up2':('up',2),
                'down2':('down',2),
               'WpTakeL':(('up',1),('left',1),2),
               'WpTakeR':(('up',1),('right',1),2),
               'BpTakeL':(('down',1),('left',1),2),
               'BpTakeR':(('down',1),('right',1),2),
               'castleR': ('right',2),
               'castleL': ('left',2)}
    
    def __init__(self, read=False, copyFrom=False, imaginary=False):
        #creates a board as a DataFrame or clones an existing board
        if imaginary:
            self.body = copyFrom.copy()
        else:
            self.body = pd.read_csv(read, index_col=0)
            
    def recon(self, rsquare):
        #look at a square and return the piece ID or False
        if not rsquare:
            return False
        if rsquare == '0':
            return False
        #print('recon rsquare: ' + str(rsquare))
        file = rsquare[0]
        rank = rsquare[1]
        square = self.body.at[rank,file]
        #print('recon square: ' + str(square))
        return square
            
    def walk(self, square, movement):
        if not square:
            return False
        file = square[0]
        rank = square[1]
        direction = movement[0]
        steps = movement[1]
        if steps == 'x':
            steps = 1
        #print(steps)
        fileCom = {"up":1, "down":-1, "left":-1, "right":1}
        global newPos
        if direction in ["left", "right"]:
            findex = board.files.index(file)
            nindex = findex + (steps * fileCom[direction])
            if nindex not in range(0,8):
                return False
            newFile = board.files[nindex]
            newPos = (newFile, rank)
        if direction in ["up", "down"]:
            newRank = (rank + (steps * fileCom[direction]))
            if newRank not in range(1,9):
                return False
            newPos = (file, newRank)
        return newPos
    
    def movePiece(self, square, newSquare):
        self.body.at[newSquare[1], newSquare[0]] = b1.body.at[square[1], square[0]]
        self.body.at[square[1], square[0]] = '0'
    
    def assessSquare(self, square, newSquare, origin):
        #print('ASSESSSQUARE')
        check = self.recon(newSquare)
        if not check:
            #out of bounds
            #print('oob')
            return 'oob'
        elif check == '0':
            #then the square is empty
            #print('empty')
            return 'empty'
        else:
            #the square is either friend or foe
            #print(recon(newSquare))
            #print(recon(square))
            if self.recon(newSquare)[0] is not self.recon(origin)[0]:
                #print('foe')
                return 'foe'
            else:
                #print('friend')
                return 'friend'
    
    def xCheck(self, x, move, square, imaginary=False):
        origin = square
        xSquares = []
        #look through each move str name, make moves and report if possible
        if 'x' in move:
            #print(move + ': x')
            #need to test further moves in this direction until blocked
            blocked = False
            #xSquares = []
            while not blocked:
                #print('not blocked')
                if len(x) == 3:
                    #move is a two-step
                    move1 = x[0]
                    #print(move1)
                    move2 = x[1]
                    #print(move2)
                    newSquare = self.walk(square, move1)
                    #print(newSquare)
                    newSquare = self.walk(newSquare, move2)
                    #print(newSquare)
                else:
                    #move is a one-step
                    newSquare = self.walk(square, x)
                    #print(newSquare)

                result = self.assessSquare(square, newSquare, origin)
                #print(result)
                if result == 'oob':
                    blocked = True
                    newSquare = False
                elif result == 'empty':
                    blocked = False
                    xSquares.append(newSquare)
                    square = newSquare
                elif result == 'friend':
                    blocked = True
                    newSquare = False
                else:
                    blocked = True
                    xSquares.append(newSquare)
                    square = newSquare
            newSquare = xSquares
            if len(newSquare) == 0:
                newSquare = False

        if 'x' not in move:
            #print(move + ': no x')
            #move only specified number of moves

            #only for castle
            if 'castle' in move:
                if 'L' in move:
                    team = 'l'
                if'R' in move:
                    team = 'r'

            #non-castle
            if len(x) == 3:
                #print(x)
                #move is a two-step
                #this is where the horses move
                move1 = x[0]
                #print(move1)
                move2 = x[1]
                #print(move2)
                newSquare = self.walk(square, move1)
                newSquare = self.walk(newSquare, move2)
            else:
                #move is a one-step
                newSquare = self.walk(square, x)
            result = self.assessSquare(square, newSquare, origin)
            if result == 'oob':
                newSquare = False
            elif result == 'friend':
                if not imaginary:
                    newSquare = False
            if 'P' in self.recon(square):
                if newSquare:
                    if not imaginary:
                        if newSquare[0] is not square[0]:
                            #if newSquare is in different file
                            if self.recon(newSquare) == '0':
                                #if newSquare is empty
                                newSquare = False
                        elif newSquare[0] is square[0]:
                            #if newSquare is in same file
                            if self.recon(newSquare) != '0':
                                #if newSquare is empty
                                newSquare = False
                    elif imaginary:
                        if newSquare[0] is square[0]:
                            #same file
                            #if self.recon(newSquare) != '0':
                                #not empty
                            newSquare = False
            if newSquare:
                xSquares.append(newSquare)
        return xSquares
    
    def moveTester(self, square, castle=True, imaginary=False):
        #returns legal moves
        #print('MOVETESTER')
        legalmoves = {}
        if self.recon(square) == '0':
            return False
        moves = self.defaultMoves(square, castle=castle)
        #print(moves)
        for move in moves:
            x = board.moveDict[move]
            #print(move)
            #print(x)
            newSquare = self.xCheck(x, move, square, imaginary)
            legalmoves[move] = newSquare

        #returns a dict of moves and their resulting squares (or False)
        return legalmoves
    
    def moveCounter(self, square, castle=True, imaginary=False):
        #print('MOVECOUNTER')
        #returns count of legal moves
        count = 0
        movesReturned = self.moveTester(square, castle=castle, imaginary=imaginary)
        #print(movesReturned)
        if movesReturned:
            for move in movesReturned.values():
                if move:
                    count += len(move)
        return count
    
    def myTeam(self, color, castle=True, imaginary=False):
        #global board
        #print('MYTEAM')
        #team is w or b (str)
        myPieces = {}
        for rank in board.ranks:
            for file in board.files:
                piece = self.body.at[rank,file]
                #print(piece)
                if color in piece:
                    movenum = self.moveCounter((file,rank), castle, imaginary)
                    myPieces[piece + str(file) + str(rank)] = ((file, rank), movenum)
        return myPieces
    
    def myMoves(self, teamdict):
        #print('MYMOVES')
        moves = {}
        for piece in teamdict:
            x = teamdict[piece]
            if x[1] != 0:
                moves[piece] = x
        return moves
    
    ### need a function that generates a long
    
    def pickRandomMove(self, team, castle=True, specific=False):
        canmoves = []
        x = self.myMoves(self.myTeam(team))
        if len(x) == 0:
            return 'done', 'done'
        #print(x)
        while len(canmoves) == 0:
            if specific:
                pick = specific
            if not specific:
                pick = random.choice(list(x))
            #print('pick: ' + str(pick))
            square = x[pick][0]
            moves = self.moveTester(square, castle)
            #print(moves)
            for y in list(moves):
                if moves[y]:
                    canmoves.append(y)
        #print(canmoves)    
        movepick = random.choice(canmoves)
        return pick, movepick
    
    def dangerDetector(self, square, team, imaginary=False):
        attacks = []
        enemy = False
        if team == 'w':
            enemy = 'b'
        if team == 'b':
            enemy = 'w'
        if not enemy:
            return False
        for rank in board.ranks:
            for file in board.files:
                if enemy in self.recon((file,rank)):
                    #print('enemy: ' + str(recon((file,rank))) + str(file) + str(rank))
                    moves = self.moveTester((file,rank), imaginary=imaginary)
                    #print(moves)
                    for move in moves.values():
                        if move:
                            #print('square: ' + str(square))
                            #print('move: ' + str(move))
                            if str(square) in str(move):
                                loc = (file,rank)
                                attacks.append(str(self.recon(loc)) + str(file) + str(rank))
        if len(attacks) == 0:
            return False
        return attacks
    
    def canCastle(self, team, side, bools):
        #check for danger or teammate along king path
        dangers = 0
        if team == 'w':
            if not bools[side]:
                    return False
            if side == 'l':
                #king is at ('e',1), rook is at ('a',1)
                #king goes to ('c',1) and left rook goes to ('d',1)
                #e1 thru c1 must not be in danger
                path = [('e',1),('d',1),('c',1)]

            if side == 'r':
                #king is at ('e',1), rook is at ('h',1)
                #king goes to ('g',1) and right rook goes to ('f',1)
                #e1 thru g1 must not be in danger
                path = [('e',1),('f',1),('g',1)]

        if team == 'b':
            if not bools[side]:
                    return False
            if side == 'l':
                #king is at ('e',8), rook is at ('a',8)
                #king goes to ('c',8) and left rook goes to ('d',8)
                #e8 thru c8 must not be in danger
                path = [('e',8),('d',8),('c',8)]

            if side == 'r':
                #king is at ('e',8), rook is at ('h',8)
                #king goes to ('g',8) and rook goes to ('f',8)
                #e8 thru g8 must not be in danger
                path = [('e',8),('f',8),('g',8)]
        for spot in path:
            piece = self.recon(spot)
            if 'K' not in piece:
                if '0' not in piece:
                    dangers += 1
                if '0' in piece:
                    x = self.dangerDetector((spot), team)
                    if x:
                        dangers += len(x)
            #print(dangers)
        if dangers == 0:
            return True
        else:
            return False
        
    def castle(team, side):
        #global board
        if team == 'w':
            if side == 'l':
                #king is at ('e',1), rook is at ('a',1)
                #king goes to ('c',1) and left rook goes to ('d',1)
                kingfrom = ('e',1)
                kingto = ('c',1)
                rookfrom = ('a',1)
                rookto = ('d',1)
                ('white queen side castle')

            if side == 'r':
                #king is at ('e',1), rook is at ('h',1)
                #king goes to ('g',1) and right rook goes to ('f',1)
                kingfrom = ('e',1)
                kingto = ('g',1)
                rookfrom = ('h',1)
                rookto = ('f',1)
                ('white king side castle')

        if team == 'b':
            if side == 'l':
                #king is at ('e',8), rook is at ('a',8)
                #king goes to ('c',8) and left rook goes to ('d',8)
                kingfrom = ('e',8)
                kingto = ('c',8)
                rookfrom = ('a',8)
                rookto = ('d',8)
                ('black queen side castle')

            if side == 'r':
                #king is at ('e',8), rook is at ('h',8)
                #king goes to ('g',8) and rook goes to ('f',8)
                kingfrom = ('e',8)
                kingto = ('g',8)
                rookfrom = ('h',8)
                rookto = ('f',8)
                print('black king side castle')
    
        ### move king ###
        self.body.at[kingto[1],kingto[0]] = self.body.at[kingfrom[1],kingfrom[0]]
        self.body.at[kingfrom[1],kingfrom[0]] = '0'
        #################
        ### move rook ###
        self.body.at[rookto[1],rookto[0]] = self.body.at[rookfrom[1],rookfrom[0]]
        self.body.at[rookfrom[1],rookfrom[0]] = '0'
        #################
        #return self.body

    def defaultMoves(self, square, castle=True):
        moves = []
        #print('DEFAULTMOVES')
        #returns default moveset for a piece
        file = square[0]
        rank = square[1]
        piece = self.recon(square)
        if piece == '0':
            return False
        color = piece[0]
        #print(color)
        pieceType = piece[1]
        #print(pieceType)
        if pieceType == 'P':
            #check pawn team and loc
            if color == 'w':
                moves = ['up1','WpTakeL','WpTakeR']
                if rank == 2:
                    #pawn can move up 1 or 2
                    moves = ['up1','up2','WpTakeL','WpTakeR']
            if color == 'b':
                moves = ['down1','BpTakeL','BpTakeR']
                if rank == 7:
                    #pawn can move down 1 or 2
                    moves = ['down1','down2','BpTakeL','BpTakeR']
        if pieceType == 'R':
            moves = ['upx','downx','leftx','rightx']
        if pieceType == 'N':
            moves = ['up2left','up2right','left2up','left2down','right2up','right2down','down2left','down2right']
        if pieceType == 'B':
            moves = ['upxleftx','upxrightx','downxleftx','downxrightx']
        if pieceType == 'Q':
            moves = ['upx','downx','leftx','rightx','upxleftx','upxrightx','downxleftx','downxrightx']
        if pieceType == 'K':
            if not castle:
                moves = ['up1','down1','left1','right1','up1left1','up1right1','down1left1','down1right1']
            if castle:
                left = self.canCastle(color, 'l', bools)
                right = self.canCastle(color,'r', bools)
                moves = ['up1','down1','left1','right1','up1left1','up1right1','down1left1','down1right1']
                if left:
                    moves.append('castleL')
                if right:
                    moves.append('castleR')
        return moves
    
    def promote(self, team, square):
        #global board
        options = ['Q', 'R', 'B', 'N']
        promotion = random.choice(options)
        newPiece = str(team) + promotion
        self.body.at[square[1],square[0]] = newPiece
        print(str(team) + 'P' + str(square[0]) + str(square[1]) + ' promoted to ' + promotion)
        #print(newPiece)
        
    def heatView(self, getSum=False):
        #https://stackoverflow.com/questions/63367517/how-to-convert-object-to-int-or-float-in-pandas
        #thanks, dad
        
        #king cannot count towards defending a square that would put it in check
        #pieces cannot count towards defending a square that would put king in check
        claim = board(imaginary=True, copyFrom=self.body)
        teamB = self.myTeam('b', imaginary=True)
        teamW = self.myTeam('w', imaginary=True)
        #print(teamB.values())

        for file in claim.files:
            for rank in claim.ranks:
                x = self.body.at[rank,file]
                if x != 0:
                    claim.body.at[rank,file] = 0
                    #print(x)
                    if 'w' in x:
                        claim.body.at[rank,file] -= 1
                    if 'b' in x:
                        claim.body.at[rank,file] += 1

        locsB = []
        locsW = []
        targetsB = []
        targetsW = []
        for loc in teamB.values():
                #print(loc[0])
                locsB.append(loc)
        for loc in teamW.values():
                #print(loc[0])
                locsW.append(loc)

        for spot in locsB:
            #use moveTester values as claims
            #print(spot[0])
            #print(square)
            claims = self.moveTester(spot[0], imaginary=True)
            #print(claims)
            #print(claims.values())
            for target in claims.values():
                targetsB += target
        #print(targets)
        print('targets: ' + str(len(targetsB)))
        for spot in locsW:
            #use moveTester values as claims
            #print(spot[0])
            #print(square)
            claims = self.moveTester(spot[0], imaginary=True)
            #print(claims)
            #print(claims.values())
            for target in claims.values():
                targetsW += target
        #print(targets)
        print('targets: ' + str(len(targetsW)))

        for b in targetsB:
            #value = game.body.at[x[1], x[0]]
            #print(value)
            claim.body.at[b[1], b[0]] += 1

        for w in targetsW:
            #value = game.body.at[x[1], x[0]]
            #print(value)
            claim.body.at[w[1], w[0]] -= 1

        x = claim.body.copy()
        x = x.apply(pd.to_numeric,errors='ignore')
        if not getSum:
            return sns.heatmap(x, cmap='binary', annot=True)
        else:
            return x.to_numpy().sum()


In [516]:
game = board("chess board castletest.csv")

# in checktest bQf5 should have 18 moves

Idea: a parallel chess board with values indicating a level of certainty that this team controls the square

In [517]:
game.body

,a,b,c,d,e,f,g,h
8,bR,bN,bB,bQ,bK,bB,bN,bR
7,bP,bP,bP,bP,bP,bP,bP,bP
6,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
2,wP,wP,wP,wP,wP,wP,wP,wP
1,wR,wN,wB,wQ,wK,0,0,wR


In [518]:
game.heatView()

TypeError: canCastle() missing 1 required positional argument: 'bools'

In [519]:
game.myTeam('b', imaginary=True)

### THESE MOVES ALL BLOCK THE WHITE KING FROM SQUARES ###

TypeError: canCastle() missing 1 required positional argument: 'bools'

In [520]:
game.moveCounter(('e',8))

TypeError: canCastle() missing 1 required positional argument: 'bools'

In [521]:
game.moveTester(('f',5))

False

In [465]:
game.myTeam('w')

{'wRa1': (('a', 1), 0),
 'wNb1': (('b', 1), 2),
 'wBc1': (('c', 1), 0),
 'wQd1': (('d', 1), 0),
 'wKe1': (('e', 1), 2),
 'wRh1': (('h', 1), 2),
 'wPa2': (('a', 2), 2),
 'wPb2': (('b', 2), 2),
 'wPc2': (('c', 2), 2),
 'wPd2': (('d', 2), 2),
 'wPe2': (('e', 2), 2),
 'wPf2': (('f', 2), 2),
 'wPg2': (('g', 2), 2),
 'wPh2': (('h', 2), 2)}

In [510]:
game.canCastle('w', 'r')

True

In [511]:
game.body

,a,b,c,d,e,f,g,h
8,bR,bN,bB,bQ,bK,bB,bN,bR
7,bP,bP,bP,bP,bP,bP,bP,bP
6,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
2,wP,wP,wP,wP,wP,wP,wP,wP
1,wR,wN,wB,wQ,wK,0,0,wR


In [500]:
game.pickRandomMove('w')

('wKe1', 'castleR')

In [40]:
game.walk(('e',2), ('up',3))

('e', 5)

In [41]:
game.recon(('e',2))

'wP'

In [42]:
game.promote('w', ('h',2))

wPh2 promoted to B


In [43]:
game.myTeam('w')

{'wRa1': (('a', 1), 0),
 'wNb1': (('b', 1), 2),
 'wBc1': (('c', 1), 0),
 'wQd1': (('d', 1), 1),
 'wBf1': (('f', 1), 0),
 'wNg1': (('g', 1), 2),
 'wRh1': (('h', 1), 0),
 'wPa2': (('a', 2), 2),
 'wPb2': (('b', 2), 2),
 'wPc2': (('c', 2), 2),
 'wPd2': (('d', 2), 1),
 'wPe2': (('e', 2), 2),
 'wPf2': (('f', 2), 2),
 'wPg2': (('g', 2), 2),
 'wBh2': (('h', 2), 3),
 'wKd3': (('d', 3), 5)}

In [512]:
def play(steps=False, resetBoard=False):
    global turn
    #global b1
    global wKmoved
    global bKmoved
    global wRmoved
    global bRmoved
    wKmoved = False
    bKmoved = False
    wMoved = {'l':False,'r':False, 'K':False}
    bMmoved = {'l':False,'r':False, 'K':False}
    count = 0
    over = False
    if resetBoard:
        game = board("chess board.csv")
        turn = 'w'
    if steps:
        step = 0
    while True: 
        ### look at team ###
        team = game.myTeam(turn)
        
        ### pick a random piece and move ###
        piece, move = game.pickRandomMove(turn)
        
        if 'P' in piece:
            pawn = True
        else:
            pawn = False
        
        if piece == 'done':
            print(piece + ' after ' + str(count) + ' turns')
            break
            
        if 'castle' not in move:
            y = game.moveDict[move]
            ### prepare the start and end positions ###
            square = team[piece][0]
            start = square
            if len(y) == 3:
                newSpace = game.walk(square, y[0])
                newSpace = game.walk(newSpace, y[1])
            else:
                newSpace = game.walk(square, y)
            end = newSpace
            
        ### castle ###
        
        if 'castle' in move:
            if 'L' in move:
                side = 'l'
            if 'R' in move:
                side = 'r'
            b1.castle(turn, side)
            if team is 'w':
                wRmoved[side] = True
                wKmoved = True
            if team is 'b':
                BRmoved[side] = True
                BKmoved = True

        ### detects if 'takes' needs to be said instead of 'to' ###
        if game.assessSquare(start, end, start) == 'foe':
            totake = ' takes '
            endpiece = str(game.recon(end))
        else:
            totake = ' to '
            endpiece = ''
            
        ### this moves a piece ###
        #print(recon(start))
        print(start)
        print(end)
        print(str(game.recon(start)) + str(start[0]) + str(start[1]) + str(totake) + str(endpiece) + str(end[0]) + str(end[1]))

        game.body.at[newSpace[1], newSpace[0]] = game.body.at[square[1], square[0]]
        game.body.at[square[1], square[0]] = '0'

        if pawn:
            if turn == 'w':
                if newSpace[1] == 8:
                    game.promote(turn, newSpace)
            if turn == 'b':
                if newSpace[1] == 1:
                    game.promote(turn, newSpace)
                    
        ### this chunk keeps track of stuff necessary for castling ###
        if game.recon(start) == 'wK':
            wKmoved = True
        if game.recon(start) == 'bK':
            bKmoved = True
        if game.recon(start) == 'wR':
            if start == ('a',1):
                wRmoved['l'] = True
            if start == ('h',1):
                wRmoved['r'] = True
        if game.recon(start) == 'bR':
            if start == ('a',8):
                bRmoved['l'] = True
            if start == ('h',8):
                bRmoved['r'] = True

        q = 0
        if turn == 'b':
            turn = 'w'
            q = 1
        if turn == 'w' and q == 0:
            turn = 'b'
        

In [513]:
play(10000, resetBoard=True)

('h', 2)
('h', 4)
wPh2 to h4
('b', 7)
('b', 6)
bPb7 to b6
('f', 2)
('f', 3)
wPf2 to f3
('b', 6)
('b', 5)
bPb6 to b5
('g', 1)
('h', 3)
wNg1 to h3
('g', 7)
('g', 5)
bPg7 to g5
('e', 2)
('e', 4)
wPe2 to e4
('c', 7)
('c', 5)
bPc7 to c5
('f', 1)
('e', 2)
wBf1 to e2
('g', 5)
('h', 4)
bPg5 takes wPh4
('f', 3)
('f', 4)
wPf3 to f4
('f', 8)
('g', 7)
bBf8 to g7


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/alessandro/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-513-5405f6b7b8a1>", line 1, in <module>
    play(10000, resetBoard=True)
  File "<ipython-input-512-e22efcab6b14>", line 21, in play
    team = game.myTeam(turn)
  File "<ipython-input-459-714c0714f3ec>", line 250, in myTeam
    movenum = self.moveCounter((file,rank), castle, imaginary)
  File "<ipython-input-459-714c0714f3ec>", line 232, in moveCounter
    movesReturned = self.moveTester(square, castle=castle, imaginary=imaginary)
  File "<ipython-input-459-714c0714f3ec>", line 216, in moveTester
    moves = self.defaultMoves(square, castle=castle)
  File "<ipython-input-459-714c0714f3ec>", line 444, in defaultMoves
    right = self.canCastle(color,'r')
  File "<ipython-input-459-714c0714f3ec>", line 347, in canCastle
    x = self.dangerDetector((spot), t

RecursionError: maximum recursion depth exceeded

In [514]:
game.body

,a,b,c,d,e,f,g,h
8,bR,bN,bB,bQ,bK,bB,bN,bR
7,bP,bP,bP,bP,bP,bP,bP,bP
6,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
2,wP,wP,wP,wP,wP,wP,wP,wP
1,wR,wN,wB,wQ,wK,0,0,wR
